In [ ]:
import sys
import os
import numpy as np

sys.path.append('..//')
from utils_mitgcm import open_mitgcm_ds_from_config
import xmitgcm as xm

In [ ]:
model = 'geneva_dummy_7ms'
dt_save = 3600
res_grid = 200

In [ ]:
def repair_coord(ds, grid_resolution):
    ds['YC'] = np.arange(1, len(ds['YC'])+1) * grid_resolution - grid_resolution/2
    ds['XC'] = np.arange(1, len(ds['XC'])+1) * grid_resolution - grid_resolution/2
    ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
    ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

    return ds

# Opening lake velocity Dataset

In [ ]:
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
ds = repair_coord(ds, res_grid)

In [ ]:
folder_path = os.path.dirname(mitgcm_config['datapath'])
output_folder = os.path.join(folder_path, "energy_budget")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
ds.UVEL.isel(time=0, Z=0).values.min()

# Opening wind stress Dataset

In [ ]:
ds_ws = xm.open_mdsdataset(
    mitgcm_config['datapath'],
    grid_dir=mitgcm_config['gridpath'],
    ref_date=mitgcm_config['ref_date'],
    prefix='wind_stress',
    delta_t=mitgcm_config['dt'],
    endian='>')

In [ ]:
ds_ws = repair_coord(ds_ws, res_grid)

In [ ]:
ds_ws.oceTAUX.isel(time=0).values

# Computing wind energy

In [ ]:
aligned_u = ds.UVEL.sel(Z=0.5, method='nearest').rename({'XG':'XC'})
aligned_u['XC'] = ds['XC']

aligned_v = ds.VVEL.sel(Z=0.5, method='nearest').rename({'YG':'YC'})
aligned_v['YC'] = ds['YC']

aligned_ocetaux = ds_ws.isel(time=0).oceTAUX.rename({'XG':'XC'})
aligned_ocetaux['XC'] = ds_ws['XC']

aligned_ocetauy = ds_ws.isel(time=0).oceTAUY.rename({'YG':'YC'})
aligned_ocetauy['YC'] = ds_ws['YC']

In [ ]:
RW = aligned_ocetaux * aligned_u.isel(time=range(24)) + aligned_ocetauy * aligned_v.isel(time=range(24))

In [ ]:
RW.isel(time=12).plot()

In [ ]:
RW_sum = RW.sum(dim=['XC','YC'])*(res_grid**2)

In [ ]:
(RW_sum).plot()

In [ ]:
full_time = aligned_u.time

In [ ]:
RW_sum_filled = RW_sum.reindex(time=full_time, fill_value=0)

In [ ]:
wind_energy_in_mj = RW_sum_filled.cumsum() * 3600 / 1e6

In [ ]:
wind_energy_in_mj.plot()

In [ ]:
df_wind_energy = wind_energy_in_mj.to_dataframe(name='wind_energy_MJ')['wind_energy_MJ'].reset_index()

In [ ]:
df_wind_energy.to_csv(os.path.join(output_folder, "wind_energy.csv"))

In [ ]:
rho_air = 1.2
C10 = 0.9 * 1e-3
W = 7
P10 =rho_air*C10*W #W/m2
A = 580*1e6 #m2
print(P10*A)

In [ ]:
(RW_sum_filled / (P10*A)).plot()